<a href="https://colab.research.google.com/github/Remdeht/ia_detector/blob/master/IA_Classification_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Irrigated Area Classification in Semi-Arid Regions**

This notebook contains an example of how to classify irrigated herbaceous and pernennial tree areas in semi arid regions. It makes use of the Google Earth Engine, for which an account is needed. For more information on the Google Earth Engine and instructions on how to make a an account click [here](https://earthengine.google.com/).  

First, Import the Github Repository containing the Irrigated Areas Classification package. 

In [ ]:
!git clone https://github.com/Remdeht/ia_detector.git ia

fatal: destination path 'ia' already exists and is not an empty directory.


In [ ]:
 !pip install monthdelta

  Created wheel for monthdelta: filename=MonthDelta-0.9.1-cp36-none-any.whl size=3111 sha256=0af59cff345dd5e0de40734091f05b17d4e04eb7a0a7f4c8887023fba0331ee4
  Stored in directory: /root/.cache/pip/wheels/5e/e9/68/c0cb0cefe6fdd49ce42a99196b272748d2d823c18c7af612cf
Successfully built monthdelta


Load the required libraries and initialize the GEE. If this is your first tume using the Earth Engine, you need to authenticate first via the ee.Authenticate() call. Instructions on how to authenticate your GEE will appear in the output below. 

In [ ]:
import ee 
ee.Authenticate()  # if first time user of the Google Earth Engine on this device, otherwise ee.Initialize() is enough

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=urUTH-906AmT6BtMYA6nlPR-WIvS7ATKYVnqu7cznFA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/0wErdHcvvba8QAWM70VoIDHnxdIwUeWxq52vlgsYBs09ryJ7ASMkQFQ

Successfully saved authorization token.


In [ ]:
import ee
ee.Initialize()

import itertools
import sys
import numpy as np
import folium
from ia.gee_functions.classification import create_features, create_training_areas, classify_irrigated_areas, \
    join_seasonal_irrigated_areas
from ia.gee_functions.constants import GEE_USER_PATH
from ia.gee_functions.export import track_task, export_to_drive
from ia.gee_functions import visualization
from datetime import datetime
from ia.gee_functions.validation import calc_area

# **Area of Interest**

Before running the classification you must select an area of interest. You can use your own area, by filling in the the Lat/Long coordinates of the rectangular extent of your choice or use one of the extents available from the list below. If you run the next code block a map will appear that allows you to check the coordinates of a location by clicking on it. 



In [ ]:
map = visualization.create_folium_map(images=None, coords=[20, 0], zoom=8, height='100%')
pu = folium.LatLngPopup().add_to(map)
map

In the dropdown menu, select which area of interest you'd wish to use for the classification. If you'd like define your own extent, select the option 'Custom Rectangular Extent'. You can then insert the maximum and minimum latitude and longitude coordinates which will then be used to create a rectangular polygon. Finally  

If you're familiar with the Google Earth Engine, and know how to upload vector layers and load them via their assetId, feel free to use your own vector asset by selecting the 'Other GEE Vector' option and inserting the assetId in the 'aoi_loc' field. 

In [ ]:
#@markdown ---
#@title Select your area of interest { run: "auto" }
area = "Campo de Cartagena" #@param ["Campo de Cartagena", "Murcia Region", "Segura Watershed", "Custom Rectangular Extent", "Other GEE Vector"] { run: "auto" }

areas = {
    'Campo de Cartagena':'cdc',
    'Murcia Region':'rdm',
    'Segura Watershed':'cds',
    'Custom Rectangular Extent':'',
    'Other GEE Vector':''
}

area_of_interest = areas[area]

if not area_of_interest == '':
  aoi = ee.FeatureCollection(f'users/Postm087/vector/outline/outline_{area_of_interest}')

#@markdown ---
#@markdown ### *Custom Rectangular Extent*
#@markdown ### Fill in maximum and minimum latitude and longitude coordinates of the extent
elif area == 'Custom Rectangular Extent':
  lat_min = 37.656830  #@param {type:"number"}
  lat_max = 37.784704  #@param {type:"number"}
  lng_min = -1.046193 #@param {type:"number"}
  lng_max = -0.832859 #@param {type:"number"}
  aoi = ee.FeatureCollection(ee.Feature(ee.Geometry.Polygon([
                   [lng_min, lat_max],
                   [lng_max, lat_max],
                   [lng_max, lat_min],
                   [lng_min, lat_min],
                   [lng_min, lat_max]
  ])))
  #@markdown ### Fill in the name of your area, this will be used to name the classification outputs { run: "auto" }
  area_of_interest = 'csp' #@param {type:"string"}
#@markdown ---
#@markdown ### *Load in your own Vector via its assetId*
elif area == 'Other GEE Vector':
  aoi_loc = ''#@param {type:"string"}
  aoi = ee.FeatureCollection(aoi_loc)
  area_of_interest = 'csp' #@param {type:"string"}


aoi_coordinates = aoi.geometry().bounds().getInfo()['coordinates']
aoi_centroid = aoi.geometry().bounds().centroid(1).coordinates().getInfo()

layer = {
    'Area of Interest':aoi.getMapId({'color':'White'})
}

map = visualization.create_folium_map(layer, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=8, height='100%')
map


Now that the area for classification has been specified, the next step before classification is to determine the timeperiod for which to perform the classification. The classification i done two times for a period of 1 year, once for the summer season which spans from April until September and once for the winter which spans from October until March. You can specify the timeperiod of your choosing in the datepicker below, but keep in mind to pick a one year period. 

In [ ]:
start_date = "2014-04-01" #@param {type:"date"}
end_date = "2015-04-01" #@param {type:"date"}


start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
year = str(int(round(np.mean([start_date_dt.year, end_date_dt.year]))))[-2:]
classification_period = {year:(start_date, end_date)}

# Some variables needed for classification but not of much importance for the user.
sat = 'landsat'  # Satellite data to use, as of now sentinel has not been tested properly so better not to use it. 
stats = ['median', 'min', 'max'] # Statistical maps to use for classification.
stats_combos = list(itertools.combinations(stats, 3))

# Some folders where the data are saved
crop_data_folder = f'{GEE_USER_PATH}/ia_classification/raster/data/{area_of_interest}/{sat}/'
training_data_folder = f'{GEE_USER_PATH}/ia_classification/raster/training_areas/{area_of_interest}/'
results_folder = f'{GEE_USER_PATH}/ia_classification/raster/results/'

Now that both the area and the time period for classification are known, the featuer data for classification can be generated from satellite imagery. For both the summer an winter season a collection of pixel statistical maps will be generated based on a selection of spectral indexes. The spectral indexes used for classification are:



1.   Normalized Difference Vegetation Index - *NDVI*
2.   Normalized Difference Water Content Index - *NDWI*
2.   Normalized Difference Water Bodies Index - *NDWI*
3.   Water-Adjusted Green Index - *WGI*
4.   Green Chlorophile Vegetation Index - *GCVI*
5.   Normalized Difference Built Up Index - *NDBI*

The generation of the feature data maps can take a while, depending on the size of the area of interest. When the task is done running on the Google Earth Engine Servers, a message will pop up in the output below. 



In [ ]:
tasks = {}
for year in classification_period:
  for season in ['winter', 'summer']:
    export_feature_task = create_features(
        classification_period[year],
        aoi,
        aoi_name=area_of_interest,
        year_string=year,
        season=season,
        collection=sat
        )
    tasks[season] = export_feature_task
track_task(tasks) # tracks the status of the export task
  

asset already exists
asset already exists
All tasks completed!


True

Let's have a look at some of the feature data layers. By running  the code block below you'll generate a folium map containing the RGB composites for the summer and the winter seasons, as well as the median NDVI values and the WGI standard deviation values per pixel over the season. The NDVI and WGI values have been visualized using a color palette ranging from red to green.   

In [ ]:
summer = ee.Image(f"{crop_data_folder}crop_data_summer_{area_of_interest}_{year}")
winter = ee.Image(f"{crop_data_folder}crop_data_winter_{area_of_interest}_{year}")
images = {
      'Summer Feature Data RGB': summer.getMapId(visualization.vis_params_rgb_ls457(bands=['red', 'green', 'blue'])), 
      'Winter Feature Data RGB': winter.getMapId(visualization.vis_params_rgb_ls457(bands=['red', 'green', 'blue'])),
      'Summer Feature Data NDVI median': summer.getMapId(visualization.vis_params_cp(band=['NDVI_median'], min_val=-1, max_val=1)), 
      'Winter Feature Data NDVI median': winter.getMapId(visualization.vis_params_cp(band=['NDVI_median'], min_val=-1, max_val=1)), 
      'Summer Feature Data WGI standard deviation': summer.getMapId(visualization.vis_params_cp(band=['WGI_std'], min_val=0, max_val=1)), 
      'Winter Feature Data WGI standard deviation': winter.getMapId(visualization.vis_params_cp(band=['WGI_std'], min_val=0, max_val=1)), 
  }
map = visualization.create_folium_map(images, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
map

#**Training Areas**

Now that the feature data is generated, the next step in to create training areas for the Random Forest thresholding. To save you the task of manually selecting these areas, a system of automatically selecting training areas for classification has been applied. The system makes use of a priori knowledge of certain land vocer classes to select training sites via thresholding.

Training sites for the following land cover classes will be generated: 

*   Forest
*   Scrubs
*   Rainfed Crops and Trees
*   Greenhouses
*   Irrigated Herbaceous Crops
*   Irrigated Perennial Trees
*   Water Bodies
*   Unproductive Areas (Urban/Fallow Lands)

By running the code block below the generation of training sites will be started. Again this may take a couple of minutes, a confirmation message will appear when the process is completed.

In [ ]:
training_tasks = {}
for year in classification_period:
  for season in ['winter', 'summer']:
    export_training_areas_task = create_training_areas(
          aoi,
          ee.Image(f"{crop_data_folder}crop_data_{season}_{area_of_interest}_{year}"),
          aoi_name=area_of_interest,
          year_string=year,
          season=season,
          )
    training_tasks[season] = export_training_areas_task
track_task(training_tasks)  # tracks the export task

Export started for training_areas/csp/training_areas_winter_csp_14
Export started for training_areas/csp/training_areas_summer_csp_14
Running task: (0 min)
Task Completed, runtime: 1 minutes


If you've reached this point, the generation of training areas should have been completed. Run the code block below to see the results. 

In [ ]:
training_classes = {
    0:'Unused',
    1:'Forest',
    2:'Scrubs',
    3:'Rainfed Crops and Trees',
    4:'Greenhouses',
    5:'Irrigated Herbaceous Crops',
    6:'Irrigated Trees',
    7:'Water Bodies',
    8:'Urban/Fallow lands'
}

summer_training = ee.Image(f'{training_data_folder}training_areas_summer_{area_of_interest}_{year}')
winter_training = ee.Image(f'{training_data_folder}training_areas_winter_{area_of_interest}_{year}')
images_training = {
      'Summer Feature Training Areas': summer_training.getMapId(visualization.vis_rf_classification(band='training')),
      'Winter Feature Training Areas': winter_training.getMapId(visualization.vis_rf_classification(band='training')) 
  }
map = visualization.create_folium_map(images_training, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
visualization.create_categorical_legend(map, visualization.vis_rf_classification()['palette'], training_classes)
map

# **Classification**

Now that the traininig sites and feature data are generated, we can finally move on to the classification. The classification is performed usin a Random Forest classifier, for which the following parameters can be set: 

*  The Number of Trees - *no_trees*
*  The Number of Variables per Split - *vps*
*  The Bagging Fraction - *bf*
*  Minimum Number of Training Points per class - *min_tp*
*  Maximum Number of Training Points per class - *max_tp*

Run the code below to start the classification. As before, this make take some time. Also, the time it takes for classification to complete is influences by the number of trees and training points specified, the lower the number the quicker the classification. 


In [ ]:
#@markdown ---
#@title ### Classification Parameters
#@markdown ### Number of Trees
no_trees = 150 #@param {type:"slider", min:0, max:500, step:10}
#@markdown ### Variables per Split
vps = 5 #@param {type:"slider", min:2, max:10, step:1}
#@markdown ### Bagging Fraction
bf = 0.65 #@param {type:"slider", min:0, max:1, step:0.05}
#@markdown ### Minimum Number of Training Points
min_tp = 1000 #@param {type:"slider", min:100, max:5000, step:100}
#@markdown ### Maximum Number of Training Points
max_tp = 5000 #@param {type:"slider", min:5000, max:75000, step:1000}
classification_tasks = {}

for year in classification_period:
  for season in ['winter', 'summer']:
    for combo in stats_combos:
      # load the feature data maps
      crop_data_image = ee.Image(f"{crop_data_folder}crop_data_{season}_{area_of_interest}_{year}")
      bands_to_select = ['red', 'green', 'blue', 'nir', 'swir1','.*std.*', 'TWI']
      stat_bands = [f'.*{s}.*' for s in list(combo)]
      bands_to_select += stat_bands # bands to select for classification
      crop_data_image = crop_data_image.select(bands_to_select)
      classification_name = "_".join(combo)
      classification_name += f'_{season}'
                  
      training_image = ee.Image(
      f'{GEE_USER_PATH}/ia_classification/raster/training_areas/{area_of_interest}/training_areas_{season}_{area_of_interest}_{year}')
      classification_task = classify_irrigated_areas(
      crop_data_image,
      training_image,
      aoi,
      classification_name,
      aoi_name=area_of_interest,
      year=year,
      clf='random_forest',
      no_trees=no_trees,
      bag_fraction=bf,
      vps=vps,
      min_tp=min_tp,
      max_tp=max_tp
      )
      classification_tasks[season]=classification_task
          
track_task(classification_tasks)

Export started for results/random_forest/csp/ia_random_forest_median_min_max_winter_150tr_5vps_65bf_csp_14
Export started for results/random_forest/csp/ia_random_forest_median_min_max_summer_150tr_5vps_65bf_csp_14
Running task: (0 min)
Running task: (1 min)
Running task: (2 min)
Task Completed, runtime: 3 minutes


True

Now that the classification has been completed, the results can be inspected 
using a folium map. 

In [ ]:
summer_clf = ee.Image(f'{results_folder}random_forest/{area_of_interest}/ia_random_forest_median_min_max_summer_{no_trees}tr_{vps}vps_{int(bf*100)}bf_{area_of_interest}_{year}')
winter_clf = ee.Image(f'{results_folder}random_forest/{area_of_interest}/ia_random_forest_median_min_max_winter_{no_trees}tr_{vps}vps_{int(bf*100)}bf_{area_of_interest}_{year}')
images_clf = {
    'Summer RF Classification results': summer_clf.getMapId(visualization.vis_rf_classification()),
    'Winter RF Classification results': winter_clf.getMapId(visualization.vis_rf_classification())
}

map = visualization.create_folium_map(images_clf, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
map = visualization.create_categorical_legend(map, visualization.vis_rf_classification()['palette'], training_classes)
map

Finally, the last step is to combine the classification results for the summer and the winter season into a single map by running the code block below.

In [ ]:
ia_summer = summer_clf.select('irrigated_area')
ia_winter = winter_clf.select('irrigated_area')
task = join_seasonal_irrigated_areas(
    ia_summer,
    ia_winter,
    area_of_interest,
    year,
    aoi_coordinates,
    export_method='asset',
  )
track_task(task)

Export started for results/irrigated_area/csp/irrigated_areas_csp_14
Running task: (0 min)
Task Completed, runtime: 1 minutes


True

# **Results**

The irrigated areas from both classification maps have been combined, creating a final map depicting the irrigated areas over the whole year. The following classes are assigned, based on which period the area was classified as irrigated. 

*  Not Irrigated

*  Year Round Irrigated Trees
*  Year Round Irrigated Crops
*  Summer Irrigated Trees
*  Summer Irrigated Crops
*  Winter Irrigated Trees
*  Winter Irrigated Crops
*  Uncertain Areas

Let's run the code below and take a look at the final result! 

In [ ]:
ia_classes = {
        0: 'Not Irrigated',
        1: 'Year Round Irrigated Trees',
        2: 'Year Round Irrigated Crops',
        3: 'Summer Irrigated Trees',
        4: 'Summer Irrigated Crops',
        5: 'Winter Irrigated Trees',
        6: 'Winter Irrigated Crops',
        7: 'Uncertain Areas',
    }


ia_year = ee.Image(f'{results_folder}irrigated_area/{area_of_interest}/irrigated_areas_{area_of_interest}_{year}').clip(aoi)

images_results = {
      'Irrigated Areas Overview': ia_year.getMapId(visualization.vis_irrigated_area_map()),
  }
map = visualization.create_folium_map(images_results, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
map = visualization.create_categorical_legend(map, visualization.vis_irrigated_area_map()['palette'], ia_classes)
map

In [ ]:
#@title Export Results to Drive {run:"auto"}
#@markdown If youd like to export the classification results to your Google drive account, please check the box below. 

export_results_to_drive = True #@param {type:"boolean"}


images_to_export = {
                    'summer_classification': summer_clf,
                    'winter_classification': winter_clf,
                    'irrigated_areas': ia_year,
                    }
                  
if export_results_to_drive:
  for img in images_to_export:
    export_task = export_to_drive(images_to_export[img], 'image', img, aoi_coordinates, 'ia_classification' )
  track_task(export_task)
 


Export started for summer_classification
Export started for winter_classification
Export started for irrigated_areas
Running Task (0 min)
Running Task (1 min)
Running Task (2 min)
Running Task (3 min)
Running Task (4 min)
Task Completed, runtime: 5 minutes
